In [1]:
#
# MultiLayerPerceptron MNIST TensorBoard
#
# @author becxer
# @email becxer87@gmail.com
# @reference https://github.com/sjchoi86/Tensorflow-101
#

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline  

print ("packages are loaded")

packages are loaded


In [2]:
# Get MNIST data set from tensorflow
from tensorflow.examples.tutorials.mnist import input_data
mnist_set = input_data.read_data_sets('images/MNIST/', one_hot = True)

train_img = mnist_set.train.images
train_label = mnist_set.train.labels
test_img = mnist_set.test.images
test_label = mnist_set.test.labels

Extracting images/MNIST/train-images-idx3-ubyte.gz
Extracting images/MNIST/train-labels-idx1-ubyte.gz
Extracting images/MNIST/t10k-images-idx3-ubyte.gz
Extracting images/MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
# Options for training
learning_rate = 0.01
training_epochs = 50
batch_size = 100
display_step = 10

# Options for Hidden Network Nodes
n_hidden = [400, 100, 30] # hidden layer's node count
w_dev = 0.1

In [4]:
# Build Graph of MultiLayer Perceptron

# Define placeholder & Variables
x = tf.placeholder("float", [None, train_img.shape[1]], name="Input_X")
y = tf.placeholder("float", [None, train_label.shape[1]], name="Output_Y")

def one_layer(_x, _W, _b, layer_name):
    with tf.variable_scope("Layer_" + str(layer_name)):
        return tf.nn.sigmoid(tf.add(tf.matmul(_x, _W),_b))

with tf.variable_scope("Weights"):
    WS = {}

with tf.variable_scope("Bias"):
    BS = {}

last_input_layer = x
last_input_layer_size = train_img.shape[1]
for idx, hl_size in enumerate(n_hidden):
    _W = tf.Variable(tf.random_normal([last_input_layer_size, hl_size], stddev=w_dev))
    _b = tf.Variable(tf.random_normal([hl_size])) 
    last_input_layer = one_layer(last_input_layer, _W, _b, idx)
    last_input_layer_size = hl_size
    WS['h_' + str(idx)] = _W
    BS['b_' + str(idx)] = _b

WS['out'] = tf.Variable(tf.random_normal([last_input_layer_size, train_label.shape[1]], stddev=w_dev))
BS['out'] = tf.Variable(tf.random_normal([train_label.shape[1]], stddev=w_dev))


# Define operators

out = one_layer(last_input_layer, WS['out'], BS['out'], 'Out')

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, y))
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

pred = tf.argmax(out, 1)
accr = tf.reduce_mean(tf.cast(tf.equal(pred, tf.argmax(y, 1)),"float"))

init = tf.initialize_all_variables()

print ("Graph build")

Graph build


In [5]:
sess = tf.Session()
sess.run(init)

# Summary initialize
tf.scalar_summary('Cost', cost)
tf.scalar_summary('Accuracy', accr)
summ = tf.merge_all_summaries()
summ_writer = tf.train.SummaryWriter('/tmp/tf_logs/mlp_mnist', graph = sess.graph)

In [6]:
# Training Graph

for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(train_img.shape[0]/batch_size)
    for i in range(num_batch):
        randidx = np.random.randint(train_img.shape[0], size=batch_size)
        batch_xs = train_img[randidx, :]
        batch_ys = train_label[randidx, :]                

        summ_batch, _ = sess.run([summ, optm], feed_dict={x: batch_xs, y: batch_ys})
        summ_writer.add_summary(summ_batch, epoch * num_batch + i)
        
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys})/num_batch
            
    if epoch % display_step == 0:
        train_img_acc = sess.run(accr , ({x: batch_xs, y: batch_ys}))
        print ("epoch: %03d/%03d  ,  cost: %.6f  ,  train_img_acc: %.3f" \
               % (epoch, training_epochs, avg_cost, train_img_acc))

print ("Training complete, Accuracy : %.6f" \
        % (sess.run(accr, {x : test_img, y : test_label}),))           

epoch: 000/050  ,  cost: 1.599856  ,  train_img_acc: 0.840
epoch: 010/050  ,  cost: 1.473836  ,  train_img_acc: 0.970
epoch: 020/050  ,  cost: 1.473252  ,  train_img_acc: 0.970
epoch: 030/050  ,  cost: 1.473207  ,  train_img_acc: 0.980
epoch: 040/050  ,  cost: 1.471846  ,  train_img_acc: 0.980
Training complete, Accuracy : 0.972100


## Run following command on shell

#### $ tensorboard --logdir=/tmp/tf_logs/mlp_mnist

        ...(syncopation)
        Starting TensorBoard  on port 6006
        (You can navigate to http://0.0.0.0:6006)
        ...(syncopation)
    
#### Open browser and go to http://0.0.0.0:6006
![tr](testimg/tensorboard.png)